In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/TFM

# List files to make sure we're in the expected directory.
# Your output will look different, showing your own Drive files here.
!ls

Mounted at /content/drive
/content/drive/My Drive/TFM
data	      file_names.okl  GAN_model     LSTM_Weimar     output.midi
example.midi  GAN_MAESTRO     LSTM_ejemplo  model_plot.png


In [ ]:
import glob
import pickle
import numpy
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import music21
from music21 import converter, instrument, note, chord, stream, pitch#manipular midis
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Bidirectional, LSTM, concatenate, Input
from tensorflow.keras.layers import BatchNormalization as BatchNorm
import tensorflow.keras.utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

La funcion get_notes(), utiliza la librería music21 para generar el dataset de notas a partir de los archivos midi. De esta forma, tendremos tres variables distintas:

- notes: todas las notas que contienen los archivos midi del conjunto de datos
- durations: las duraciones de cada una de esas notas.
- Offsets: distancia entre la última nota tocada y la nueva.


# Preparación de datos.
La funcion get_notes(), utiliza la librería music21 para generar el dataset de notas a partir de los archivos midi. De esta forma, tendremos cuatro variables distintas:

- notes: todas las notas que contienen los archivos midi del conjunto de datos
- durations: las duraciones de cada una de esas notas.
- Offsets: distancia entre la última nota tocada y la nueva.
- Chords: Acorde que acompaña a cada una de las notas.

Antes de eso, reduciremos los acordes a mayores y menores y ajustaremos la resolución del tiempo a 1/8 de pulso. Después, separaremos los datos en secuencias sucesivas de longitud 100 y utilizaremos el 90% para entrenar y el 10% para test.

Como ya tenemos los datos resultantes, nos ahorraremos el proceso y llos cargaremos directamente. Aún así, pondremos las funciones que se utilizaron.

In [ ]:
# Procesado acordes, los reduciremos a mayores o menores.
def proc_chords(chords):
  chord_dict = {'A#': 'Bb','B#': 'C','Cb': 'B','C#': 'Db',
    'D#': 'Eb','E#': 'F','Fb': 'E','F#': 'Gb','G#': 'Ab',
    'A#m': 'Bbm','B#m': 'Cm','Cbm': 'Bm','C#m': 'Dbm','D#m': 'Ebm',
    'E#m': 'Fm','Fbm': 'Em','F#m': 'Gbm','G#m': 'Abm', 'NC': 'N'}

  for i, ch in enumerate(chords): # eliminamos séptimas etc
    ch_type = ''
    if len(ch) < 2:
      tone = ch[0]
    elif ch[1] in ['#', 'b']:
      tone = ch[:2]
    else:
      tone = ch[:1]
    if  'm' in ch or '-' in ch:
      ch_type = 'm'

    chords[i] = tone+ch_type

  chords = [chord_dict.get(chord, chord) for chord in chords] # eliminamos la redundancia (Bb = A#)

  return chords

In [ ]:
def round_length(number): # funcion para redondear los tiempos
  values_list = [0 , 1/8,  1/4, 3/8, 1/2, 5/8, 6/8,7/8, 1]

  entero = int(number)
  decim = number - entero
  decim = min(values_list, key=lambda x: abs(x - decim))

  number = entero + decim
  return number

In [ ]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    offsets = []
    durations = []
    chords = []

    filenames = glob.glob('LSTM_Weimar/Weimar_csv/*.csv')
    for file in filenames:

            mel = pd.read_csv(file)

            print("Parsing %s" % file)

            if mel.signature[0] == '"4/4"':

              notes = numpy.append(notes, mel.pitch.values)
              mel['onset'] =  mel.onset.values/mel.beat_duration.median() # el onset y duracion estan en segundos, los pasamos a beats.
              mel['onset'] = mel['onset'].apply(round_length)
              mel['duration'] =  mel.duration.values/mel.beat_duration.median()
              mel['duration'] = mel['duration'].apply(round_length)
              mel['duration'] = numpy.where(mel['duration'] > 8, 8, mel['duration']) # las duraciones mayores de un compas las acortamos a un compas
              onset = mel.onset.values - mel.onset.shift(1).values
              onset[0] = 0
              onset = numpy.where(onset > 8, 8, onset)
              offsets = numpy.append(offsets, onset)
              durations = numpy.append(durations, mel.duration.values)
              processed_chords = proc_chords(mel.chord.values)
              chords = numpy.append(chords, processed_chords)
              print("Parsed %s" % file)
            else:
              print(file, 'not 4/4')

    # for i in range(len(offsets)): #redondeamos los valroes
    #         offsets[i] = round_length(offsets[i])

    with open('LSTM_Weimar/data_csv_aug/notes_aug', 'wb') as filepath:
        pickle.dump(notes, filepath)

    with open('LSTM_Weimar/data_csv_aug/durations_aug', 'wb') as filepath:
        pickle.dump(durations, filepath)

    with open('LSTM_Weimar/data_csv_aug/offsets_aug', 'wb') as filepath:
        pickle.dump(offsets, filepath)

    with open('LSTM_Weimar/data_csv_aug/chords_aug', 'wb') as filepath:
        pickle.dump(chords, filepath)

    return notes, offsets, durations, chords

In [ ]:
# notes, offsets, durations, chords = get_notes()

open_file = open('LSTM_ejemplo/data_csv_aug/notes_aug', "rb")
notes = pickle.load(open_file)
open_file.close()

open_file = open('LSTM_ejemplo/data_csv_aug/offsets_aug', "rb")
offsets = pickle.load(open_file)
open_file.close()

open_file = open('LSTM_ejemplo/data_csv_aug/durations_aug', "rb")
durations = pickle.load(open_file)
open_file.close()

open_file = open('LSTM_ejemplo/data_csv_aug/chords_aug', "rb")
chords = pickle.load(open_file)
open_file.close()

### Data augmentation
Vamos a aumentar el dataset transponiendo las melodias y los acordes 2 semitonos arriba y abajo como en https://arxiv.org/pdf/2008.01307.pdf

El dataset resultante ya lo tenemos guardado. Más adelante cargaremos los datos ya preprocesados, así que comentaremos esta parte.

In [ ]:
def transponer_acordes(acordes, semitonos):
    notas = numpy.array(["C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B"])
    for i in range(len(acordes)):
      for j in range(len(notas)):
        if notas[j] in acordes[i]:
          notes_trans = numpy.roll(notas, -semitonos)
          acordes[i] = acordes[i].replace(notas[j], notes_trans[j])
          break

    return acordes

### Secuencias
Con prepare_sequences() separamos los datos en secuencias de longitud, **sequence_lentgh**, y para cada secuencia tenemos una etiqueta, que será el elemento sequence_lentgh + 1. De esta manera nuestro modelo utilizará los primeros *sequence_lentgh* elementos para predecir el siguiente.

En concreto, las secuencias de entrada serán vectores normalizados. En cuanto al output, lo representará como una variable categórica con one hot encoding.


In [ ]:
def prepare_sequences(notes, n_vocab):
	""" Prepare the sequences used by the Neural Network """
	sequence_length = 100

	# get all pitch names
	pitchnames = sorted(set(item for item in notes))

	 # create a dictionary to map pitches to integers
	note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

	network_input = []
	network_output = []

	# create input sequences and the corresponding outputs
	for i in range(0, len(notes) - sequence_length, 1):
		sequence_in = notes[i:i + sequence_length]
		sequence_out = notes[i + sequence_length]
		network_input.append([note_to_int[char] for char in sequence_in])
		network_output.append(note_to_int[sequence_out])

	n_patterns = len(network_input)

	# reshape the input into a format compatible with LSTM layers
	network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
	# normalize input
	network_input = network_input / float(n_vocab)

	network_output = np_utils.to_categorical(network_output)

	return (network_input, network_output)

# Modelo.

Comenzamos definiendo el modelo.



In [ ]:
def create_network(network_input_notes, n_vocab_notes, network_input_Chords, n_vocab_Chords, network_input_offsets, n_vocab_offsets, network_input_durations, n_vocab_durations):

	# Branch of the network that considers notes
	inputNotesLayer = Input(shape=(network_input_notes.shape[1], network_input_notes.shape[2]))
	inputNotes = LSTM(
		256,
		input_shape=(network_input_notes.shape[1], network_input_notes.shape[2]),
		return_sequences=True
	)(inputNotesLayer)
	inputNotes = Dropout(0.2)(inputNotes)

	# Branch of the network that considers chords
	inputChordsLayer = Input(shape=(network_input_Chords.shape[1], network_input_Chords.shape[2]))
	inputChords = LSTM(
		256,
		input_shape=(network_input_Chords.shape[1], network_input_Chords.shape[2]),
		return_sequences=True
	)(inputChordsLayer)
	inputChords = Dropout(0.2)(inputChords)

	# Branch of the network that considers note offset
	inputOffsetsLayer = Input(shape=(network_input_offsets.shape[1], network_input_offsets.shape[2]))
	inputOffsets = LSTM(
		256,
		input_shape=(network_input_offsets.shape[1], network_input_offsets.shape[2]),
		return_sequences=True
	)(inputOffsetsLayer)
	inputOffsets = Dropout(0.2)(inputOffsets)

	# Branch of the network that considers note duration
	inputDurationsLayer = Input(shape=(network_input_durations.shape[1], network_input_durations.shape[2]))
	inputDurations = LSTM(
		256,
		input_shape=(network_input_durations.shape[1], network_input_durations.shape[2]),
		return_sequences=True
	)(inputDurationsLayer)
	#inputDurations = Dropout(0.3)(inputDurations)
	inputDurations = Dropout(0.2)(inputDurations)

	#Concatentate the four input networks together into one branch now
	inputs = concatenate([inputNotes, inputChords, inputOffsets, inputDurations])

	# A cheeky LSTM to consider everything learnt from the four separate branches
	x = LSTM(512, return_sequences=True)(inputs)
	x = Dropout(0.3)(x)
	x = LSTM(512)(x)
	x = BatchNorm()(x)
	x = Dropout(0.3)(x)
	x = Dense(256, activation='relu')(x)

	#Time to split into four branches again...

	# Branch of the network that classifies the note
	outputNotes = Dense(128, activation='relu')(x)
	outputNotes = BatchNorm()(outputNotes)
	outputNotes = Dropout(0.3)(outputNotes)
	outputNotes = Dense(n_vocab_notes, activation='softmax', name="Note")(outputNotes)

  # Branch of the network that classifies the note
	outputChords = Dense(128, activation='relu')(x)
	outputChords = BatchNorm()(outputChords)
	outputChords = Dropout(0.3)(outputChords)
	outputChords = Dense(n_vocab_Chords, activation='softmax', name="Chord")(outputChords)

	# Branch of the network that classifies the note offset
	outputOffsets = Dense(128, activation='relu')(x)
	outputOffsets = BatchNorm()(outputOffsets)
	outputOffsets = Dropout(0.3)(outputOffsets)
	outputOffsets = Dense(n_vocab_offsets, activation='softmax', name="Offset")(outputOffsets)

	# Branch of the network that classifies the note duration
	outputDurations = Dense(128, activation='relu')(x)
	outputDurations = BatchNorm()(outputDurations)
	outputDurations = Dropout(0.3)(outputDurations)
	outputDurations = Dense(n_vocab_durations, activation='softmax', name="Duration")(outputDurations)

	# Tell Keras what our inputs and outputs are
	model = Model(inputs=[inputNotesLayer, inputChordsLayer,inputOffsetsLayer, inputDurationsLayer], outputs=[outputNotes, outputChords, outputOffsets, outputDurations])

	#Adam seems to be faster than RMSProp and learns better too
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# Useful to try RMSProp though

	# LOAD WEIGHTS HERE IF YOU WANT TO CONTINUE TRAINING!
	model.load_weights('LSTM_ejemplo/modelo_csv_aug/weights-improvement-68-4.1149-bigger.hdf5')

	return model

# Entrenamiento.

Defininimos la funciónde entrenamiento

In [ ]:
def train_network(notes, chords, offsets, durations):

    n_vocab_notes = len(set(notes))

    n_vocab_chords = len(set(chords))

    n_vocab_offsets = len(set(offsets))

    n_vocab_durations = len(set(durations))

    path = 'LSTM_ejemplo/data_csv_aug/'
    # Cargamos las variables de entrenamiento
    train_input_notes = numpy.load(path + 'train_input_notes.npy')
    train_output_notes = numpy.load(path + 'train_output_notes.npy')
    train_input_durations = numpy.load(path + 'train_input_durations.npy')
    train_output_durations = numpy.load(path + 'train_output_durations.npy')
    train_input_offsets = numpy.load(path + 'train_input_offsets.npy')
    train_output_offsets = numpy.load(path + 'train_output_offsets.npy')
    train_input_chords = numpy.load(path + 'train_input_chords.npy')
    train_output_chords = numpy.load(path + 'train_output_chords.npy')

    # Cargamos los datos de prueba
    test_input_notes = numpy.load(path + 'test_input_notes.npy')
    test_output_notes = numpy.load(path + 'test_output_notes.npy')
    test_input_durations = numpy.load(path + 'test_input_durations.npy')
    test_output_durations = numpy.load(path + 'test_output_durations.npy')
    test_input_offsets = numpy.load(path + 'test_input_offsets.npy')
    test_output_offsets = numpy.load(path + 'test_output_offsets.npy')
    test_input_chords = numpy.load(path + 'test_input_chords.npy')
    test_output_chords = numpy.load(path + 'test_output_chords.npy')

    train_input = [train_input_notes, train_input_chords, train_input_offsets, train_input_durations]
    train_output = [train_output_notes, train_output_chords, train_output_offsets, train_output_durations]
    test_input = [test_input_notes, test_input_chords, test_input_offsets, test_input_durations]
    test_output = [test_output_notes, test_output_chords, test_output_offsets, test_output_durations]

    model = create_network(train_input_notes, n_vocab_notes, train_input_chords, n_vocab_chords, train_input_offsets, n_vocab_offsets, train_input_durations, n_vocab_durations)
    model.summary()
    train(model, train_input, train_output, test_input, test_output)

In [ ]:
class SaveHistoryCallback(Callback):
    def __init__(self, file_path):
        super(SaveHistoryCallback, self).__init__()
        self.file_path = file_path
        self.history_list = []

    def on_epoch_end(self, epoch, logs=None):
        # Append the current epoch's training and validation metrics to the history list
        self.history_list.append(logs)

        # Save the history list to the file using pickle
        with open(self.file_path, 'wb') as file:
            pickle.dump(self.history_list, file)

In [ ]:
def train(model, train_input, train_output, test_input, test_output):
	""" train the neural network """
	filepath = "LSTM_Weimar/modelo_csv_aug/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
	checkpoint = ModelCheckpoint(
		filepath,
		monitor='loss',
		verbose=0,
		save_best_only=True,
		mode='min'
	)

	custom_callback = SaveHistoryCallback(file_path='LSTM_ejemplo/modelo_csv_aug/history.pkl')

	callbacks_list = [checkpoint, custom_callback]


	model.fit(train_input, train_output, epochs=300, initial_epoch = 68, batch_size=64, callbacks=callbacks_list, verbose=1, validation_data = (test_input, test_output))

In [ ]:
if __name__ == '__main__':
	train_network(notes, chords, offsets, durations)

# Generación

Para la generación de nuevas melodías se utiliza un input formado por una secuencia de notas, acordes, duraciones y offsets, cada una de distinta secuencia original. De esta forma la secuencia resultante es diferente a cualquiera ya existente. Utilizando esta nueva secuencia se generaran las nuevas notas.

Comenzamos cargando los datos y definiendo las funciones que utilizamos.

In [ ]:
""" Generate a piano midi file """
#load the notes used to train the model
open_file = open('LSTM_ejemplo/data_csv_aug/notes_aug', "rb")
notes = pickle.load(open_file)
open_file.close()

open_file = open('LSTM_ejemplo/data_csv_aug/offsets_aug', "rb")
offsets = pickle.load(open_file)
open_file.close()

open_file = open('LSTM_ejemplo/data_csv_aug/durations_aug', "rb")
durations = pickle.load(open_file)
open_file.close()

open_file = open('LSTM_ejemplo/data_csv_aug/chords_aug', "rb")
chords = pickle.load(open_file)
open_file.close()

In [ ]:
def prepare_sequences(notes, pitchnames, n_vocab):
	""" Prepare the sequences used by the Neural Network """
	# map between notes and integers and back
	note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

	sequence_length = 100
	network_input = []
	output = []
	for i in range(0, len(notes) - sequence_length, 1):
		sequence_in = notes[i:i + sequence_length]
		sequence_out = notes[i + sequence_length]
		network_input.append([note_to_int[char] for char in sequence_in])
		output.append(note_to_int[sequence_out])

	n_patterns = len(network_input)

	# reshape the input into a format compatible with LSTM layers
	normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
	# normalize input
	normalized_input = normalized_input / float(n_vocab)

	return (network_input, normalized_input)

Definimos las funciones para transformar los vectores a archivos midi

In [ ]:
def get_chord_notes(ch):
	if 'm' in ch:
		ch = ch.replace('m', '', 1)
		mid_pitch = pitch.Pitch(ch).midi
		ch = [mid_pitch, mid_pitch + 3, mid_pitch + 7]

	else:
		mid_pitch = pitch.Pitch(ch).midi
		ch = [mid_pitch, mid_pitch + 4, mid_pitch + 7]

	return chord.Chord(ch)


def create_midi(pred_output):
	""" convert the output from the prediction to notes and create a midi file
		from the notes """
	offset = 0
	output_notes = []
	output_chords = []
	ch = 'N'
   # todas las columnas se convierten a string, los transformmos
	notes = pred_output[0].astype(float).astype(int).tolist()
	chords = pred_output[1].tolist()
	durations = pred_output[2].astype(float).tolist()
	offsets = pred_output[3].astype(float).tolist()


	print("---")
	print("Creating Midi File...")

	# create note and chord objects based on the values generated by the model
	x = 0 # this is the counter

	for pattern in notes:
		offset += offsets[x]
		new_note = note.Note(pattern)
		new_note.offset = offset
		new_note.duration.quarterLength = durations[x]
		output_notes.append(new_note)

		# for chords
		if (chords[x] != ch) & (chords[x] != 'N'): # cuando cambia el acorde
			if ch != 'N': #añadimos el viejo acorde tras el cambio. La duraacion se extendera hasta el cambio de acorde.
				new_chord.offset = chord_offset
				new_chord.duration.quarterLength = offset - chord_offset
				output_chords.append(new_chord)
				# print(new_chord.offset)

			new_chord = get_chord_notes(chords[x]) #añadimos lo que sera el futuro nuevo acorde
			chord_offset = offset
			ch = chords[x]

		if x == len(notes)-1:
			if ch != 'N': #añadimos el viejo acorde tras el cambio. La duraacion se extendera hasta el cambio de acorde.
				new_chord.offset = chord_offset
				new_chord.duration.quarterLength = offset - chord_offset
				output_chords.append(new_chord)

		# increase offset each iteration so that notes do not stack
		x = x+1

	stream_notes = stream.Stream(output_notes)

	stream_chords = stream.Stream(output_chords)
	stream_notes.insert(instrument.Flute())
	stream_chords.insert(instrument.Piano())

	midi_stream = stream.Stream()
	midi_stream.insert(music21.tempo.MetronomeMark(number = 90))
	midi_stream.insert(0,stream_notes)
	midi_stream.insert(0,stream_chords)

	midi_stream.write('midi', fp='LSTM_ejemplo/ejemplo_Weimar_CSV_4.mid')

	print("Midi created!")
	return midi_stream

In [ ]:
def generate(notes, chords, offsets, durations):

	notenames = sorted(set(item for item in notes))
	n_vocab_notes = len(set(notes))
	network_input_notes, normalized_input_notes = prepare_sequences(notes, notenames, n_vocab_notes)

	offsetnames = sorted(set(item for item in offsets))
	n_vocab_offsets = len(set(offsets))
	network_input_offsets, normalized_input_offsets = prepare_sequences(offsets, offsetnames, n_vocab_offsets)

	durationames = sorted(set(item for item in durations))
	n_vocab_durations = len(set(durations))
	network_input_durations, normalized_input_durations = prepare_sequences(durations, durationames, n_vocab_durations)

	chordnames = sorted(set(item for item in chords))
	n_vocab_chords = len(set(chords))
	network_input_chords, normalized_input_chords = prepare_sequences(chords, chordnames, n_vocab_chords)

	model = create_network(normalized_input_notes, n_vocab_notes, normalized_input_chords, n_vocab_chords, normalized_input_offsets, n_vocab_offsets, normalized_input_durations, n_vocab_durations)

	prediction_output = generate_notes(model, network_input_notes, network_input_chords, network_input_offsets, network_input_durations, notenames, chordnames, offsetnames, durationames, n_vocab_notes, n_vocab_chords, n_vocab_offsets, n_vocab_durations)
	create_midi(prediction_output)



def generate_notes(model, network_input_notes, network_input_chords, network_input_offsets, network_input_durations, notenames, chordnames, offsetnames, durationames, n_vocab_notes, n_vocab_chords, n_vocab_offsets, n_vocab_durations):
	""" Generate notes from the neural network based on a sequence of notes """
	# pick a random sequence from the input as a starting point for the prediction
	start = numpy.random.randint(0, len(network_input_notes)-1)
	start2 = numpy.random.randint(0, len(network_input_offsets)-1)
	start3 = numpy.random.randint(0, len(network_input_durations)-1)
	# start4 = numpy.random.randint(0, len(network_input_chords)-1)

	int_to_note = dict((number, note) for number, note in enumerate(notenames))
	note_to_int = {v: k for k, v in int_to_note.items()}
	int_to_offset = dict((number, note) for number, note in enumerate(offsetnames))
	offset_to_int = {v: k for k, v in int_to_offset.items()}
	int_to_duration = dict((number, note) for number, note in enumerate(durationames))
	duration_to_int = {v: k for k, v in int_to_duration.items()}
	int_to_chord = dict((number, note) for number, note in enumerate(chordnames))
	chord_to_int = {v: k for k, v in int_to_chord.items()}

	pattern = network_input_notes[start]
	pattern2 = network_input_offsets[start2]
	pattern3 = network_input_durations[start3]
	pattern4 = network_input_chords[start]

	prediction_output = []

	# generate notes or chords
	for note_index in range(100):
		note_prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
		predictedNote = note_prediction_input[-1][-1][-1]
		note_prediction_input = note_prediction_input / float(n_vocab_notes)

		offset_prediction_input = numpy.reshape(pattern2, (1, len(pattern2), 1))
		offset_prediction_input = offset_prediction_input / float(n_vocab_offsets)

		duration_prediction_input = numpy.reshape(pattern3, (1, len(pattern3), 1))
		duration_prediction_input = duration_prediction_input / float(n_vocab_durations)

		chord_prediction_input = numpy.reshape(pattern4, (1, len(pattern4), 1))
		chord_prediction_input = chord_prediction_input / float(n_vocab_chords)

		prediction = model.predict([note_prediction_input, chord_prediction_input, offset_prediction_input, duration_prediction_input], verbose=0)

		index = numpy.argmax(prediction[0])
		result = int_to_note[index]

		chord = numpy.argmax(prediction[1])
		chord_result = int_to_chord[chord]

		offset = numpy.argmax(prediction[2])
		offset_result = int_to_offset[offset]

		duration = numpy.argmax(prediction[3])
		duration_result = int_to_duration[duration]


		print("Next note: " + str(int_to_note[predictedNote]) + " - Chord: " + str(int_to_chord[chord]) + " - Duration: " + str(int_to_duration[duration]) + " - Offset: " + str(int_to_offset[offset]))


		prediction_output.append([result, chord_result, offset_result, duration_result])

		pattern.append(index)
		pattern2.append(offset)
		pattern3.append(duration)
		pattern4.append(chord)
		pattern = pattern[1:len(pattern)]
		pattern2 = pattern2[1:len(pattern2)]
		pattern3 = pattern3[1:len(pattern3)]
		pattern4 = pattern4[1:len(pattern4)]

	return numpy.transpose(prediction_output)

### ejecutar

In [ ]:
if __name__ == '__main__':
	generate(notes, chords, offsets, durations)